In [4]:
import numpy as np
import common
from common import reset_counters, check_mat, randomize_matrix
from multiplication import strassen
# from lu_factorization import recursive_lu
from functions import LU_factorization

# Import counters for easy access
counter_add = common.counter_add
counter_mul = common.counter_mul
counter_sub = common.counter_sub


## Recursive LU Factorization Test


In [5]:
# Test recursive LU factorization with a simple 2x2 matrix
A = np.array([[2.0, 1.0], [1.0, 2.0]])
reset_counters()

L, U = LU_factorization(A, strassen)

print("Original matrix A:")
print(A)
print("\nL (lower triangular):")
print(L)
print("\nU (upper triangular):")
print(U)
print("\nVerification (L * U should equal A):")
LU = strassen(L, U)
print(LU)
print("\nDifference:")
print(np.abs(A - LU))
print("\nIs correct?", np.allclose(A, LU, atol=1e-10))
print("\nCounters:")
print(f"Additions: {common.counter_add}, Multiplications: {common.counter_mul}, Subtractions: {common.counter_sub}")


Original matrix A:
[[2. 1.]
 [1. 2.]]

L (lower triangular):
[[1.  0. ]
 [0.5 1. ]]

U (upper triangular):
[[2.  1. ]
 [0.  1.5]]

Verification (L * U should equal A):
[[2. 1.]
 [1. 2.]]

Difference:
[[0. 0.]
 [0. 0.]]

Is correct? True

Counters:
Additions: 17, Multiplications: 23, Subtractions: 0


In [6]:
# Test with a 4x4 matrix
A = np.array([[4.0, 1.0, 2.0, 0.5],
              [1.0, 3.0, 0.5, 1.0],
              [2.0, 0.5, 5.0, 1.0],
              [0.5, 1.0, 1.0, 2.0]])

reset_counters()

L, U = LU_factorization(A, strassen)

print("Original matrix A:")
print(A)
print("\nL (lower triangular):")
print(L)
print("\nU (upper triangular):")
print(U)
print("\nVerification (L * U should equal A):")
LU = strassen(L, U)
print(LU)
print("\nDifference (should be close to zero):")
diff = np.abs(A - LU)
print(diff)
print(f"\nMax difference: {np.max(diff):.2e}")
print("\nIs correct?", np.allclose(A, LU, atol=1e-9))

# Verify structure
L_lower = np.allclose(np.triu(L, k=1), 0)
L_unit = np.allclose(np.diag(L), 1.0)
U_upper = np.allclose(np.tril(U, k=-1), 0)
print(f"\nL is lower triangular: {L_lower}")
print(f"L has unit diagonal: {L_unit}")
print(f"U is upper triangular: {U_upper}")

print("\nCounters:")
print(f"Additions: {common.counter_add}, Multiplications: {common.counter_mul}, Subtractions: {common.counter_sub}")


Original matrix A:
[[4.  1.  2.  0.5]
 [1.  3.  0.5 1. ]
 [2.  0.5 5.  1. ]
 [0.5 1.  1.  2. ]]

L (lower triangular):
[[1.         0.         0.         0.        ]
 [0.25       1.         0.         0.        ]
 [0.5        0.         1.         0.        ]
 [0.125      0.31818182 0.1875     1.        ]]

U (upper triangular):
[[4.         1.         2.         0.5       ]
 [0.         2.75       0.         0.875     ]
 [0.         0.         4.         0.75      ]
 [0.         0.         0.         1.51846591]]

Verification (L * U should equal A):
[[4.  1.  2.  0.5]
 [1.  3.  0.5 1. ]
 [2.  0.5 5.  1. ]
 [0.5 1.  1.  2. ]]

Difference (should be close to zero):
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

Max difference: 0.00e+00

Is correct? True

L is lower triangular: True
L has unit diagonal: True
U is upper triangular: True

Counters:
Additions: 182, Multiplications: 268, Subtractions: 6


## Verify recursive LU factorization for various matrix sizes


In [7]:
GREEN = '\033[92m'
RED = '\033[91m'
ENDCOLOR = '\033[0m'

test_count = 20
correct_count = 0

for n in range(1, test_count + 1):
    print(f"Verifying n = {n}")
    
    # Generate a random invertible matrix
    # Use a matrix with larger diagonal values to ensure invertibility
    A = np.random.rand(n, n) * 0.5 + np.eye(n) * 2.0
    
    reset_counters()
    
    try:
        L, U = LU_factorization(A, strassen)
        
        # Verify that L * U = A
        LU = strassen(L, U)
        if check_mat(LU, A, tol=1e-5):
            # Verify structure
            L_lower = np.allclose(np.triu(L, k=1), 0)
            L_unit = np.allclose(np.diag(L), 1.0)
            U_upper = np.allclose(np.tril(U, k=-1), 0)
            
            if L_lower and L_unit and U_upper:
                correct_count += 1
            else:
                print(f"  Warning: Structure incorrect for n={n}")
        else:
            print(f"  Warning: L * U != A for n={n}")
    except Exception as e:
        print(f"  Error for n={n}: {e}")

print(f'\n{correct_count}/{test_count} correct')
if correct_count == test_count:
    print(f'{GREEN}Success!{ENDCOLOR}')
else:
    print(f'{RED}Test failed!{ENDCOLOR}')


Verifying n = 1
Verifying n = 2
Verifying n = 3
Verifying n = 4
Verifying n = 5
Verifying n = 6
Verifying n = 7
Verifying n = 8
Verifying n = 9
Verifying n = 10
Verifying n = 11
Verifying n = 12
Verifying n = 13
Verifying n = 14
Verifying n = 15
Verifying n = 16
Verifying n = 17
Verifying n = 18
Verifying n = 19
Verifying n = 20

20/20 correct
Success!


## Compare with NumPy's LU factorization


In [8]:
# Note: NumPy uses partial pivoting (permutation matrix P), so results will differ
# But both should satisfy: P * A = L * U (NumPy) and A = L * U (our recursive)

from scipy.linalg import lu as scipy_lu

A = np.array([[4.0, 1.0, 2.0, 0.5],
              [1.0, 3.0, 0.5, 1.0],
              [2.0, 0.5, 5.0, 1.0],
              [0.5, 1.0, 1.0, 2.0]])

print("Original matrix A:")
print(A)

print("\n" + "="*60)
print("Recursive LU (no pivoting):")
print("="*60)
reset_counters()
L_rec, U_rec = LU_factorization(A, strassen)
print("L:")
print(L_rec)
print("\nU:")
print(U_rec)
print("\nVerification: L * U =")
print(strassen(L_rec, U_rec))
print(f"\nOperations: Add={common.counter_add}, Mul={common.counter_mul}, Sub={common.counter_sub}")

print("\n" + "="*60)
print("NumPy/Scipy LU (with partial pivoting):")
print("="*60)
P_np, L_np, U_np = scipy_lu(A)
print("P (permutation matrix):")
print(P_np)
print("\nL:")
print(L_np)
print("\nU:")
print(U_np)
print("\nVerification: P * A = L * U")
print("P * A:")
print(P_np @ A)
print("\nL * U:")
print(L_np @ U_np)

Original matrix A:
[[4.  1.  2.  0.5]
 [1.  3.  0.5 1. ]
 [2.  0.5 5.  1. ]
 [0.5 1.  1.  2. ]]

Recursive LU (no pivoting):
L:
[[1.         0.         0.         0.        ]
 [0.25       1.         0.         0.        ]
 [0.5        0.         1.         0.        ]
 [0.125      0.31818182 0.1875     1.        ]]

U:
[[4.         1.         2.         0.5       ]
 [0.         2.75       0.         0.875     ]
 [0.         0.         4.         0.75      ]
 [0.         0.         0.         1.51846591]]

Verification: L * U =
[[4.  1.  2.  0.5]
 [1.  3.  0.5 1. ]
 [2.  0.5 5.  1. ]
 [0.5 1.  1.  2. ]]

Operations: Add=182, Mul=268, Sub=6

NumPy/Scipy LU (with partial pivoting):
P (permutation matrix):
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]

L:
[[1.         0.         0.         0.        ]
 [0.25       1.         0.         0.        ]
 [0.5        0.         1.         0.        ]
 [0.125      0.31818182 0.1875     1.        ]]

U:
[[4.         1.         2.    

## Test with larger matrices


In [9]:
import time

test_sizes = [8, 16, 32]

for n in test_sizes:
    print(f"\nTesting {n}x{n} matrix:")
    print("-" * 50)
    
    # Generate a random matrix
    np.random.seed(42)
    A = np.random.rand(n, n) * 0.5 + np.eye(n) * 2.0
    
    reset_counters()
    start_time = time.time()
    
    try:
        L, U = LU_factorization(A, strassen)
        elapsed_time = time.time() - start_time
        
        # Verify
        LU = strassen(L, U)
        max_diff = np.max(np.abs(A - LU))
        is_correct = np.allclose(A, LU, atol=1e-9)
        
        print(f"Time: {elapsed_time*1000:.2f} ms")
        print(f"Max difference: {max_diff:.2e}")
        print(f"Is correct: {is_correct}")
        print(f"Operations: Add={common.counter_add}, Mul={common.counter_mul}, Sub={common.counter_sub}")
        
        # Verify structure
        L_lower = np.allclose(np.triu(L, k=1), 0)
        L_unit = np.allclose(np.diag(L), 1.0)
        U_upper = np.allclose(np.tril(U, k=-1), 0)
        print(f"Structure: L_lower={L_lower}, L_unit={L_unit}, U_upper={U_upper}")
        
    except Exception as e:
        print(f"Error: {e}")



Testing 8x8 matrix:
--------------------------------------------------
Time: 5.04 ms
Max difference: 5.55e-17
Is correct: True
Operations: Add=1836, Mul=2708, Sub=48
Structure: L_lower=True, L_unit=True, U_upper=True

Testing 16x16 matrix:
--------------------------------------------------
Time: 25.23 ms
Max difference: 4.44e-16
Is correct: True
Operations: Add=16440, Mul=24352, Sub=264
Structure: L_lower=True, L_unit=True, U_upper=True

Testing 32x32 matrix:
--------------------------------------------------
Time: 191.92 ms
Max difference: 4.44e-16
Is correct: True
Operations: Add=138672, Mul=206384, Sub=1248
Structure: L_lower=True, L_unit=True, U_upper=True
